In [37]:
with open('file.txt', 'r') as file:
    lines = file.read().strip().split('\n')



In [38]:
# === Parse the machine specifications ===

import re

def parse_machine(line):
    """Parse a machine line into target state and buttons"""
    # Extract indicator lights pattern [.##.]
    lights_match = re.search(r'\[(.*?)\]', line)
    lights_str = lights_match.group(1)
    target = [1 if c == '#' else 0 for c in lights_str]
    
    # Extract all button specifications (0,1,2)
    buttons = []
    button_matches = re.findall(r'\(([0-9,]+)\)', line)
    for button_str in button_matches:
        indices = [int(x) for x in button_str.split(',')]
        buttons.append(indices)
    
    return target, buttons


In [39]:
# === Gaussian Elimination over GF(2) with Minimum Solution ===

import numpy as np
from itertools import product

def solve_gf2(target, buttons):
    """
    Solve the system of linear equations over GF(2) to find minimum button presses.
    
    We want to find x (button presses) such that:
    A * x = target (mod 2)
    
    where A is a matrix where A[i][j] = 1 if button j toggles light i
    """
    n_lights = len(target)
    n_buttons = len(buttons)
    
    # Create the augmented matrix [A | target]
    matrix = []
    for light_idx in range(n_lights):
        row = []
        for button in buttons:
            row.append(1 if light_idx in button else 0)
        row.append(target[light_idx])
        matrix.append(row)
    
    matrix = np.array(matrix, dtype=int)
    
    # Reduced Row Echelon Form (RREF) in GF(2)
    rows, cols = matrix.shape
    pivot_cols = []
    current_row = 0
    
    for col in range(n_buttons):
        # Find pivot
        pivot_row = None
        for row in range(current_row, rows):
            if matrix[row, col] == 1:
                pivot_row = row
                break
        
        if pivot_row is None:
            continue
        
        # Swap rows
        if pivot_row != current_row:
            matrix[[current_row, pivot_row]] = matrix[[pivot_row, current_row]]
        
        pivot_cols.append(col)
        
        # Eliminate all other 1s in this column (forward AND backward)
        for row in range(rows):
            if row != current_row and matrix[row, col] == 1:
                matrix[row] = (matrix[row] + matrix[current_row]) % 2
        
        current_row += 1
    
    # Check for inconsistency
    for row in range(current_row, rows):
        if matrix[row, -1] == 1:
            return None  # No solution
    
    # Identify free variables (non-pivot columns)
    free_vars = [col for col in range(n_buttons) if col not in pivot_cols]
    
    # If no free variables, there's a unique solution
    if not free_vars:
        solution = [0] * n_buttons
        for row, col in enumerate(pivot_cols):
            solution[col] = matrix[row, -1]
        return solution
    
    # Try all combinations of free variables to find minimum solution
    min_presses = float('inf')
    best_solution = None
    
    for free_values in product([0, 1], repeat=len(free_vars)):
        solution = [0] * n_buttons
        
        # Set free variables
        for i, col in enumerate(free_vars):
            solution[col] = free_values[i]
        
        # Calculate pivot variables based on free variables
        for row, col in enumerate(pivot_cols):
            val = matrix[row, -1]
            for j in range(col + 1, n_buttons):
                val = (val + matrix[row, j] * solution[j]) % 2
            solution[col] = val
        
        # Count total presses
        presses = sum(solution)
        if presses < min_presses:
            min_presses = presses
            best_solution = solution
    
    return best_solution


In [34]:
# === Solve for all machines ===

total_presses = 0

print("Solving all machines...\n")

for i, line in enumerate(lines):
    target, buttons = parse_machine(line)
    solution = solve_gf2(target, buttons)
    
    if solution:
        presses = sum(solution)
        total_presses += presses
        
        if i < 5:  # Show first 5
            print(f"Machine {i+1}: {presses} presses")
    else:
        print(f"Machine {i+1}: NO SOLUTION!")

print(f"\n{'='*60}")
print(f"ANSWER: Total button presses = {total_presses}")
print(f"{'='*60}")

Solving all machines...

Machine 1: 3 presses
Machine 2: 1 presses
Machine 3: 1 presses
Machine 4: 3 presses
Machine 5: 6 presses

ANSWER: Total button presses = 538


In [40]:
# === Part 2: Parse joltage requirements ===

def parse_machine_part2(line):
    """Parse machine line to extract buttons and joltage requirements"""
    # Extract all button specifications (0,1,2)
    buttons = []
    button_matches = re.findall(r'\(([0-9,]+)\)', line)
    for button_str in button_matches:
        indices = [int(x) for x in button_str.split(',')]
        buttons.append(indices)
    
    # Extract joltage requirements {3,5,4,7}
    joltage_match = re.search(r'\{([0-9,]+)\}', line)
    joltage_str = joltage_match.group(1)
    target_joltage = [int(x) for x in joltage_str.split(',')]
    
    return buttons, target_joltage


In [41]:
import pulp

def solve_ilp_joltage(buttons, target_joltage):
    """
    Solve for minimum button presses using Integer Linear Programming.
    
    Minimize: sum of all button presses (x1 + x2 + ... + xn)
    Subject to: A * x = target_joltage, where x >= 0 (integers)
    """
    n_counters = len(target_joltage)
    n_buttons = len(buttons)
    
    # Create ILP problem
    prob = pulp.LpProblem("ButtonPresses", pulp.LpMinimize)
    
    # Decision variables: number of times to press each button
    x = [pulp.LpVariable(f"button_{i}", lowBound=0, cat='Integer') 
         for i in range(n_buttons)]
    
    # Objective: minimize total button presses
    prob += pulp.lpSum(x), "Total_Presses"
    
    # Constraints: each counter must reach its target value
    for counter_idx in range(n_counters):
        counter_sum = pulp.lpSum([x[btn_idx] for btn_idx, button in enumerate(buttons) 
                                  if counter_idx in button])
        prob += counter_sum == target_joltage[counter_idx], f"Counter_{counter_idx}"
    
    # Solve
    prob.solve(pulp.PULP_CBC_CMD(msg=0))
    
    # Extract solution
    if prob.status == pulp.LpStatusOptimal:
        return [int(var.varValue) for var in x]
    else:
        return None

print("✓ ILP solver ready")

✓ ILP solver ready


In [42]:
# === Solve Part 2 for all machines ===

total_presses_part2 = 0

print("Solving Part 2 for all machines...\n")

for i, line in enumerate(lines):
    buttons, joltage = parse_machine_part2(line)
    solution = solve_ilp_joltage(buttons, joltage)
    
    if solution:
        presses = sum(solution)
        total_presses_part2 += presses
        
        if i < 5:  # Show first 5
            print(f"Machine {i+1}: {presses} presses")
    else:
        print(f"Machine {i+1}: NO SOLUTION!")

print(f"\n{'='*60}")
print(f"PART 2 ANSWER: Total button presses = {total_presses_part2}")
print(f"{'='*60}")

Solving Part 2 for all machines...

Machine 1: 78 presses
Machine 2: 180 presses
Machine 3: 36 presses
Machine 4: 85 presses
Machine 5: 199 presses

PART 2 ANSWER: Total button presses = 20298
